# CPSC4810 Team Project 
#### Leo Liang
#### Qirui Cao
#### Latesh Subramanayam

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## Section 1 Data Inspection and Cleaning

(description data sources for each dataset:)

### Inspect the county data set

In [2]:
dfCounty = pd.read_csv("data/us_county.csv")

In [3]:
dfCounty.head(5)

fips          county    state state_code    male  female  median_age  \
0  1001  Autauga County  Alabama         AL   26874   28326        37.8   
1  1003  Baldwin County  Alabama         AL  101188  106919        42.8   
2  1005  Barbour County  Alabama         AL   13697   12085        39.9   
3  1007     Bibb County  Alabama         AL   12152   10375        39.9   
4  1009   Blount County  Alabama         AL   28434   29211        40.8   

   population  female_percentage        lat       long  
0       55200          51.315217  32.534923 -86.642730  
1      208107          51.376936  30.727479 -87.722564  
2       25782          46.873788  31.869581 -85.393210  
3       22527          46.055844  32.998628 -87.126475  
4       57645          50.673953  33.980869 -86.567380

In [4]:
print("County dataset has", dfCounty.shape[0], "rows and", dfCounty.shape[1], "columns in the dataset.")

County dataset has 3220 rows and 11 columns in the dataset.


In [5]:
print("The number of rows in the dataset is equal to the number of unique values in column 'fips':", 
      len(dfCounty) == dfCounty.fips.nunique())

The number of rows in the dataset is equal to the number of unique values in column 'fips': True


Since the number of rows of the dataframe is equal to the number of unique values for variable "fips". We can consider the column "fips" as the key of the dataframe. That is, each row in dataframe dfCounty represents a different US county. 

In [6]:
dfCounty.isnull().sum()

fips                  0
county                0
state                 0
state_code           79
male                  0
female                0
median_age            0
population            0
female_percentage     0
lat                   0
long                  0
dtype: int64

Then, by running the total number of missing values for each variable, we find out that there are 79 missing values for "state_code". Then we need to check those rows containing missing values. 

In [7]:
dfCounty[dfCounty.state_code.isnull()]['state'].value_counts()

Puerto Rico             78
District of Columbia     1
Name: state, dtype: int64

In [8]:
print("Puerto Rico has", len(dfCounty[dfCounty.state == "Puerto Rico"]), "rows in the dataframe. ")

Puerto Rico has 78 rows in the dataframe. 


In [9]:
print("District of Columbia", len(dfCounty[dfCounty.state == "District of Columbia"]), "rows in the dataframe")

District of Columbia 1 rows in the dataframe


We find out that the only two states which do not have a state code are Puerto Rico and District of Columbia. By comparing the number of rows for two states, and the number of missing value occurance, we can be sure that state Puerto Rico and District of Columbia do not have state code. 

This is because both areas are not a US state, then they do not have a US state code. Since Puerto Rico is not part of 50 states + 1 disctrict, we exclude those observations from the project. 

In [10]:
dfCounty = dfCounty.drop(dfCounty[dfCounty.state == "Puerto Rico"].index)
dfCounty.state.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

Then, we aggregate the population by all states to replicate the raw dataframe with state-level information.  

In [11]:
state_pop = pd.DataFrame(dfCounty.groupby('state')[['male','female','population']].sum())
state_pop["female_proportion"] = state_pop.female/state_pop.population
state_pop.reset_index(level=0, inplace=True)
state_pop

state      male    female  population  female_proportion
0                Alabama   2355799   2508881     4864680           0.515734
1                 Alaska    385579    352937      738516           0.477900
2                Arizona   3453439   3493246     6946685           0.502865
3               Arkansas   1468412   1522259     2990671           0.509002
4             California  19453769  19694991    39148760           0.503081
5               Colorado   2780831   2750310     5531141           0.497241
6            Connecticut   1747131   1834373     3581504           0.512180
7               Delaware    459570    489925      949495           0.515985
8   District of Columbia    324881    359617      684498           0.525373
9                Florida  10071925  10526214    20598139           0.511027
10               Georgia   5012229   5285255    10297484           0.513257
11                Hawaii    713772    708257     1422029           0.498061
12                 Idaho    845564    842245     1687809           0.499017
13              Illinois   6295915   6525582    12821497           0.508956
14               Indiana   3271068   3366358     6637426           0.507178
15                  Iowa   1554481   1578018     3132499           0.503757
16                Kansas   1449413   1459363     2908776           0.501710
17              Kentucky   2186715   2253489     4440204           0.507519
18             Louisiana   2279968   2383648     4663616           0.511116
19                 Maine    652299    680514     1332813           0.510585
20              Maryland   2910323   3093112     6003435           0.515224
21         Massachusetts   3313979   3516214     6830193           0.514804
22              Michigan   4899118   5058370     9957488           0.507997
23             Minnesota   2751748   2775610     5527358           0.502159
24           Mississippi   1450379   1538383     2988762           0.514722
25              Missouri   2988772   3101290     6090062           0.509238
26               Montana    524220    517512     1041732           0.496780
27              Nebraska    949910    954850     1904760           0.501297
28                Nevada   1467165   1455684     2922849           0.498036
29         New Hampshire    664983    678639     1343622           0.505082
30            New Jersey   4335930   4545915     8881845           0.511821
31            New Mexico   1035850   1056584     2092434           0.504955
32              New York   9519555  10098898    19618453           0.514765
33        North Carolina   4944759   5210865    10155624           0.513101
34          North Dakota    386183    366018      752201           0.486596
35                  Ohio   5703967   5937912    11641879           0.510048
36              Oklahoma   1941576   1976561     3918137           0.504464
37                Oregon   2023021   2058922     4081943           0.504398
38          Pennsylvania   6263189   6527992    12791181           0.510351
39          Rhode Island    513332    543279     1056611           0.514171
40        South Carolina   2405726   2550199     4955925           0.514576
41          South Dakota    435761    428528      864289           0.495816
42             Tennessee   3243678   3407411     6651089           0.512309
43                 Texas  13849775  14035420    27885195           0.503329
44                  Utah   1532768   1512582     3045350           0.496686
45               Vermont    307973    317004      624977           0.507225
46              Virginia   4138574   4275200     8413774           0.508119
47            Washington   3645665   3648671     7294336           0.500206
48         West Virginia    904196    924858     1829054           0.505648
49             Wisconsin   2872376   2906018     5778394           0.502911
50               Wyoming    296979    284857      581836           0.489583

### Inspect the Poverty data 

In [12]:
dfPoverty = pd.read_csv("data/PovertyEstimates_us_county.csv")

In [13]:
dfPoverty.head(5)

FIPStxt Stabr       Area_name  Rural-urban_Continuum_Code_2003  \
0        0    US   United States                              NaN   
1     1000    AL         Alabama                              NaN   
2     1001    AL  Autauga County                              2.0   
3     1003    AL  Baldwin County                              4.0   
4     1005    AL  Barbour County                              6.0   

   Urban_Influence_Code_2003  Rural-urban_Continuum_Code_2013  \
0                        NaN                              NaN   
1                        NaN                              NaN   
2                        2.0                              2.0   
3                        5.0                              3.0   
4                        6.0                              6.0   

   Urban_Influence_Code_2013  POVALL_2018  CI90LBAll_2018  CI90UBALL_2018  \
0                        NaN     41852315        41619366        42085264   
1                        NaN       801758          785668          817848   
2                        2.0         7587            6334            8840   
3                        2.0        21069           17390           24748   
4                        6.0         6788            5662            7914   

   ...  CI90UB517P_2018  MEDHHINC_2018  CI90LBINC_2018  CI90UBINC_2018  \
0  ...             17.2          61937           61843           62031   
1  ...             23.7          49881           49123           50639   
2  ...             23.9          59338           53628           65048   
3  ...             16.9          57588           54437           60739   
4  ...             45.9          34382           31157           37607   

   POV04_2018  CI90LB04_2018  CI90UB04_2018  PCTPOV04_2018  CI90LB04P_2018  \
0   3758704.0      3714862.0      3802546.0           19.5            19.3   
1     73915.0        69990.0        77840.0           26.0            24.6   
2         NaN            NaN            NaN            NaN             NaN   
3         NaN            NaN            NaN            NaN             NaN   
4         NaN            NaN            NaN            NaN             NaN   

   CI90UB04P_2018  
0            19.7  
1            27.4  
2             NaN  
3             NaN  
4             NaN  

[5 rows x 34 columns]

In [14]:
dfPoverty.columns

Index(['FIPStxt', 'Stabr', 'Area_name', 'Rural-urban_Continuum_Code_2003',
       'Urban_Influence_Code_2003', 'Rural-urban_Continuum_Code_2013',
       'Urban_Influence_Code_2013', 'POVALL_2018', 'CI90LBAll_2018',
       'CI90UBALL_2018', 'PCTPOVALL_2018', 'CI90LBALLP_2018',
       'CI90UBALLP_2018', 'POV017_2018', 'CI90LB017_2018', 'CI90UB017_2018',
       'PCTPOV017_2018', 'CI90LB017P_2018', 'CI90UB017P_2018', 'POV517_2018',
       'CI90LB517_2018', 'CI90UB517_2018', 'PCTPOV517_2018', 'CI90LB517P_2018',
       'CI90UB517P_2018', 'MEDHHINC_2018', 'CI90LBINC_2018', 'CI90UBINC_2018',
       'POV04_2018', 'CI90LB04_2018', 'CI90UB04_2018', 'PCTPOV04_2018',
       'CI90LB04P_2018', 'CI90UB04P_2018'],
      dtype='object')

Then, we altered the dataset to keep the relevant columns.

In [15]:
dfPoverty = dfPoverty[['FIPStxt', 'Stabr', 'Area_name', 'Rural-urban_Continuum_Code_2013', 'Urban_Influence_Code_2013', 
                        'POVALL_2018', 'PCTPOVALL_2018', 'POV017_2018', 'PCTPOV017_2018', 'MEDHHINC_2018']]

In [16]:
print("Poverty dataset has", dfPoverty.shape[0], "rows and", dfPoverty.shape[1], "columns in the dataset.")

Poverty dataset has 3193 rows and 10 columns in the dataset.


In [17]:
print("The number of rows in the dataset is equal to the number of unique values in column 'fips':", 
      len(dfPoverty) == dfPoverty["FIPStxt"].nunique())

The number of rows in the dataset is equal to the number of unique values in column 'fips': True


Similarly, since the number of rows of the dataframe is equal to the number of unique values for variable "FIPStxt". We can consider the column "FIPStxt" as the key of the dataframe. That is, each row in dataframe dfCounty represents a different US area. 

In [18]:
dfPoverty.isnull().sum()

FIPStxt                             0
Stabr                               0
Area_name                           0
Rural-urban_Continuum_Code_2013    52
Urban_Influence_Code_2013          52
POVALL_2018                         0
PCTPOVALL_2018                      0
POV017_2018                         0
PCTPOV017_2018                      0
MEDHHINC_2018                       0
dtype: int64

Then, by running the total number of missing values for each variable, we find out that there are 52 missing values for 2013's "RUCC" and "UIC". Then we need to check those rows containing missing values.

In [19]:
dfPoverty[dfPoverty["Rural-urban_Continuum_Code_2013"].isna()]["Area_name"]

0              United States
1                    Alabama
69                    Alaska
99                   Arizona
115                 Arkansas
191               California
250                 Colorado
315              Connecticut
324                 Delaware
328     District of Columbia
330                  Florida
398                  Georgia
558                   Hawaii
563                    Idaho
608                 Illinois
711                  Indiana
804                     Iowa
904                   Kansas
1010                Kentucky
1131               Louisiana
1196                   Maine
1213                Maryland
1238           Massachusetts
1253                Michigan
1337               Minnesota
1425             Mississippi
1508                Missouri
1624                 Montana
1681                Nebraska
1775                  Nevada
1793           New Hampshire
1804              New Jersey
1826              New Mexico
1860                New York
1923          

In [20]:
dfPoverty[dfPoverty["Urban_Influence_Code_2013"].isna()]["Area_name"]

0              United States
1                    Alabama
69                    Alaska
99                   Arizona
115                 Arkansas
191               California
250                 Colorado
315              Connecticut
324                 Delaware
328     District of Columbia
330                  Florida
398                  Georgia
558                   Hawaii
563                    Idaho
608                 Illinois
711                  Indiana
804                     Iowa
904                   Kansas
1010                Kentucky
1131               Louisiana
1196                   Maine
1213                Maryland
1238           Massachusetts
1253                Michigan
1337               Minnesota
1425             Mississippi
1508                Missouri
1624                 Montana
1681                Nebraska
1775                  Nevada
1793           New Hampshire
1804              New Jersey
1826              New Mexico
1860                New York
1923          

By observing the missing values in both columns, the corresponding area names are states and US itself. The missing values are due to the fact that the country US itself and states are not applied to the county-level code, as it only applies to counties. 

Meanwhile, Rural-urban Continuum Code 2013 and Urban Influence Code 2013 are all categorical variables, which have their corresponding categories. (Please see documents below) Then, we need to map the code to their corresponding description. 

Rural-Urban Continuum Code: https://www.ers.usda.gov/data-products/rural-urban-continuum-codes/documentation/
Urban Influence Code: https://www.ers.usda.gov/data-products/urban-influence-codes/documentation.aspx.

In [21]:
RUCC = {1: "Level 1", 
           2: "Level 2",
           3: "Level 3",
           4: "Level 4",
           5: "Level 5",
           6: "Level 6", 
           7: "Level 7", 
           8: "Level 8",
           9: "Level 9"
          }

UIC = {1: "In large metro area of 1+ million residents",
       2: "In small metro area of less than 1 million residents",
       3: "Micropolitan area adjacent to large metro area",
       4: "Noncore adjacent to large metro area",
       5: "Micropolitan area adjacent to small metro area",
       6: "Noncore adjacent to small metro area and contains a town of at least 2,500 residents",
       7: "Noncore adjacent to small metro area and does not contain a town of at least 2,500 residents",
       8: "Micropolitan area not adjacent to a metro area",
       9: "Noncore adjacent to micro area and contains a town of at least 2,500 residents",
       10: "Noncore adjacent to micro area and does not contain a town of at least 2,500 residents",
       11: "Noncore not adjacent to metro or micro area and contains a town of at least 2,500 residents",
       12: "Noncore not adjacent to metro or micro area and does not contain a town of at least 2,500 residents"
      }

dfPoverty["Rural-urban_Continuum_Code_2013"] = dfPoverty["Rural-urban_Continuum_Code_2013"].map(RUCC)
dfPoverty["Urban_Influence_Code_2013"] = dfPoverty["Urban_Influence_Code_2013"].map(UIC)

In [22]:
dfPoverty.head(5)

FIPStxt Stabr       Area_name Rural-urban_Continuum_Code_2013  \
0        0    US   United States                             NaN   
1     1000    AL         Alabama                             NaN   
2     1001    AL  Autauga County                         Level 2   
3     1003    AL  Baldwin County                         Level 3   
4     1005    AL  Barbour County                         Level 6   

                           Urban_Influence_Code_2013  POVALL_2018  \
0                                                NaN     41852315   
1                                                NaN       801758   
2  In small metro area of less than 1 million res...         7587   
3  In small metro area of less than 1 million res...        21069   
4  Noncore adjacent to small metro area and conta...         6788   

   PCTPOVALL_2018  POV017_2018  PCTPOV017_2018  MEDHHINC_2018  
0            13.1     12997532            18.0          61937  
1            16.8       255613            23.9          49881  
2            13.8         2509            19.3          59338  
3             9.8         6442            13.9          57588  
4            30.9         2242            43.9          34382

In [23]:
dfPoverty.describe()

FIPStxt   POVALL_2018  PCTPOVALL_2018   POV017_2018  \
count   3193.000000  3.193000e+03     3193.000000  3.193000e+03   
mean   30356.223301  3.932257e+04       15.125274  1.221190e+04   
std    15178.992102  7.583740e+05        6.092508  2.358859e+05   
min        0.000000  5.000000e+00        2.600000  5.000000e+00   
25%    18173.000000  1.621000e+03       10.800000  5.030000e+02   
50%    29175.000000  3.876000e+03       14.100000  1.200000e+03   
75%    45075.000000  9.874000e+03       18.300000  2.966000e+03   
max    56045.000000  4.185232e+07       54.000000  1.299753e+07   

       PCTPOV017_2018  MEDHHINC_2018  
count     3193.000000    3193.000000  
mean        21.045443   52944.895709  
std          8.862190   13878.755803  
min          2.500000   25385.000000  
25%         14.500000   43783.000000  
50%         20.100000   50748.000000  
75%         26.200000   59020.000000  
max         68.300000  140382.000000

### Inspect COVID case and death data

In [24]:
dfcovCnty = pd.read_csv("data/covid_us_county.csv")

In [25]:
dfcovCnty.head(5)

fips   county    state        lat       long        date  cases  \
0  1001.0  Autauga  Alabama  32.539527 -86.644082  2020-01-22      0   
1  1001.0  Autauga  Alabama  32.539527 -86.644082  2020-01-23      0   
2  1001.0  Autauga  Alabama  32.539527 -86.644082  2020-01-24      0   
3  1001.0  Autauga  Alabama  32.539527 -86.644082  2020-01-25      0   
4  1001.0  Autauga  Alabama  32.539527 -86.644082  2020-01-26      0   

  state_code  deaths  
0         AL       0  
1         AL       0  
2         AL       0  
3         AL       0  
4         AL       0

In [26]:
print("COVID dataset has", dfcovCnty.shape[0], "rows and", dfcovCnty.shape[1], "columns in the dataset.")

COVID dataset has 1858152 rows and 9 columns in the dataset.


This data contains the county-level cumulative cases and deaths starting from January 22nd, 2020. Let's take the current date, and inspect the missing values

In [27]:
dfcovCnty_current = dfcovCnty[dfcovCnty['date'] == '2021-07-30']
dfcovCnty_current.head()

fips   county    state        lat       long        date  cases  \
555   1001.0  Autauga  Alabama  32.539527 -86.644082  2021-07-30   7479   
1111  1003.0  Baldwin  Alabama  30.727750 -87.722071  2021-07-30  24499   
1667  1005.0  Barbour  Alabama  31.868263 -85.387129  2021-07-30   2462   
2223  1007.0     Bibb  Alabama  32.996421 -87.125115  2021-07-30   2845   
2779  1009.0   Blount  Alabama  33.982109 -86.567906  2021-07-30   7266   

     state_code  deaths  
555          AL     114  
1111         AL     328  
1667         AL      61  
2223         AL      65  
2779         AL     139

In [28]:
print("current date COVID dataset has", dfcovCnty_current.shape[0], "rows and", dfcovCnty_current.shape[1], "columns in the dataset.")

current date COVID dataset has 3342 rows and 9 columns in the dataset.


In [29]:
dfcovCnty_current.isnull().sum()

fips          10
county         6
state          0
lat            0
long           0
date           0
cases          0
state_code    89
deaths         0
dtype: int64

There are 10 missing values in column "fips", 6 in column "county" and 89 in column "state_code". We need to check all the rows containing these missing values. 

In [30]:
dfcovCnty_current[dfcovCnty_current.fips.isnull()]

fips                                     county          state  \
706119    NaN                        Dukes and Nantucket  Massachusetts   
726691    NaN     Federal Correctional Institution (FCI)       Michigan   
744483    NaN  Michigan Department of Corrections (MDOC)       Michigan   
886263    NaN                                Kansas City       Missouri   
1644091   NaN                                 Bear River           Utah   
1646871   NaN                               Central Utah           Utah   
1657435   NaN                             Southeast Utah           Utah   
1657991   NaN                             Southwest Utah           Utah   
1659659   NaN                                  TriCounty           Utah   
1664107   NaN                               Weber-Morgan           Utah   

               lat        long        date  cases state_code  deaths  
706119   41.406747  -70.687635  2021-07-30   3009         MA       6  
726691    0.000000    0.000000  2021-07-30    320         MI       5  
744483    0.000000    0.000000  2021-07-30  26770         MI     155  
886263   39.099700  -94.578600  2021-07-30  49320         MO     617  
1644091  41.521068 -113.083282  2021-07-30  23219         UT     114  
1646871  39.372319 -111.575868  2021-07-30   9650         UT      72  
1657435  38.996171 -110.701396  2021-07-30   3975         UT      31  
1657991  37.854472 -111.441876  2021-07-30  31376         UT     289  
1659659  40.124915 -109.517442  2021-07-30   5053         UT      42  
1664107  41.271160 -111.914512  2021-07-30  33602         UT     233

Since in the merging stage of our project, we will merge by fips, which is the key column for most of our county-level dataframe, we will remove those rows containing missing values in fips. 

In [31]:
dfcovCnty_current[dfcovCnty_current.county.isnull()]

fips county                     state      lat      long  \
57267       60.0    NaN            American Samoa -14.2710 -170.1320   
188483   88888.0    NaN          Diamond Princess   0.0000    0.0000   
318587   99999.0    NaN            Grand Princess   0.0000    0.0000   
319143      66.0    NaN                      Guam  13.4443  144.7937   
1180943     69.0    NaN  Northern Mariana Islands  15.0979  145.6739   
1673559     78.0    NaN            Virgin Islands  18.3358  -64.8963   

               date  cases state_code  deaths  
57267    2021-07-30      0        NaN       0  
188483   2021-07-30     49        NaN       0  
318587   2021-07-30    103        NaN       3  
319143   2021-07-30   8541        NaN     143  
1180943  2021-07-30    183        NaN       2  
1673559  2021-07-30   4606        NaN      37

Since the missing values in county indicates two cruise ships and four areas outside 50 states + 1 district, we remove those rows as well. 

In [32]:
dfcovCnty=dfcovCnty.dropna(subset=['fips','county'])
dfcovCnty_current=dfcovCnty_current.dropna(subset=['fips','county'])

Then, we checked the rows with missing state codes, and found out that only Puerto Rico and District of Columbia don't have state code. It is the same case as the previous situation where both areas are not a US state, thus there are no state codes. And we removed rows with Puerto Rico as well.

In [33]:
dfcovCnty_current[dfcovCnty_current.state_code.isnull()].state.value_counts()

Puerto Rico             80
District of Columbia     3
Name: state, dtype: int64

In [34]:
print("Puerto Rico has", len(dfcovCnty_current[dfcovCnty_current.state == "Puerto Rico"]), "rows in the dataframe. ")

Puerto Rico has 80 rows in the dataframe. 


In [35]:
print("District of Columbia", len(dfcovCnty_current[dfcovCnty_current.state == "District of Columbia"]), "rows in the dataframe")

District of Columbia 3 rows in the dataframe


In [36]:
dfcovCnty = dfcovCnty.drop(dfcovCnty[dfcovCnty.state == "Puerto Rico"].index)
dfcovCnty_current = dfcovCnty[dfcovCnty.date == '2021-07-30']
dfcovCnty.state.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [37]:
print("Now, the number of rows in the dataset is equal to the number of unique values in column 'fips':", 
      len(dfcovCnty_current) == dfcovCnty_current.fips.nunique())

Now, the number of rows in the dataset is equal to the number of unique values in column 'fips': True


Since the number of rows of the dataframe is equal to the number of unique values for variable "fips". We can consider the column "fips" as the key of the dataframe. That is, each row in dataframe current represents a different US county.

In [38]:
dfcovCnty_current.describe()

fips          lat         long         cases        deaths
count   3246.000000  3246.000000  3246.000000  3.246000e+03   3246.000000
mean   32083.336106    37.260840   -89.428180  1.065934e+04    187.518792
std    17742.211826     8.522405    20.596120  3.834142e+04    723.357640
min     1001.000000     0.000000  -174.159600  0.000000e+00      0.000000
25%    19041.500000    34.263872   -98.062361  1.020000e+03     18.000000
50%    30046.000000    38.162993   -89.900575  2.568500e+03     49.000000
75%    47008.500000    41.685722   -82.841249  6.731750e+03    117.000000
max    90056.000000    69.314792     0.000000  1.297087e+06  24691.000000

Similar to the county data, we also aggregated the cumulative cases and deaths by state level to replicate the dataframe. 

In [39]:
dfcovSt = pd.DataFrame(dfcovCnty.groupby(["state", "date"])[["cases", "deaths"]].sum())
dfcovSt.reset_index(inplace=True)
dfcovSt.head()

state        date  cases  deaths
0  Alabama  2020-01-22      0       0
1  Alabama  2020-01-23      0       0
2  Alabama  2020-01-24      0       0
3  Alabama  2020-01-25      0       0
4  Alabama  2020-01-26      0       0

In [40]:
dfcovSt_current = dfcovSt[dfcovSt.date == "2021-07-30"]
dfcovSt_current

state        date    cases  deaths
555                 Alabama  2021-07-30   577463   11510
1111                 Alaska  2021-07-30    75486     389
1667                Arizona  2021-07-30   925169   18224
2223               Arkansas  2021-07-30   385113    6123
2779             California  2021-07-30  3956303   64423
3335               Colorado  2021-07-30   575082    6945
3891            Connecticut  2021-07-30   354335    8293
4447               Delaware  2021-07-30   111275    1830
5003   District of Columbia  2021-07-30    50398    1149
5559                Florida  2021-07-30  2634234   39079
6115                Georgia  2021-07-30  1179449   21676
6671                 Hawaii  2021-07-30    41925     537
7227                  Idaho  2021-07-30   200484    2197
7783               Illinois  2021-07-30  1419611   25873
8339                Indiana  2021-07-30   771299   14005
8895                   Iowa  2021-07-30   378831    6183
9451                 Kansas  2021-07-30   333850    5261
10007              Kentucky  2021-07-30   482599    7334
10563             Louisiana  2021-07-30   541679   10999
11119                 Maine  2021-07-30    70372     899
11675              Maryland  2021-07-30   467961    9818
12231         Massachusetts  2021-07-30   716771   18076
12787              Michigan  2021-07-30   984016   21028
13343             Minnesota  2021-07-30   612701    7761
13899           Mississippi  2021-07-30   343505    7543
14455              Missouri  2021-07-30   634482    9464
15011               Montana  2021-07-30   116366    1704
15567              Nebraska  2021-07-30   228450    2280
16123                Nevada  2021-07-30   356401    5912
16679         New Hampshire  2021-07-30   100657    1387
17235            New Jersey  2021-07-30  1038171   26595
17791            New Mexico  2021-07-30   210416    4410
18347              New York  2021-07-30  2147318   53632
18903        North Carolina  2021-07-30  1048076   13635
19459          North Dakota  2021-07-30   111570    1570
20015                  Ohio  2021-07-30  1127808   20492
20571              Oklahoma  2021-07-30   480635    7485
21127                Oregon  2021-07-30   219755    2858
21683          Pennsylvania  2021-07-30  1229002   27850
22239          Rhode Island  2021-07-30   154339    2740
22795        South Carolina  2021-07-30   617148    9904
23351          South Dakota  2021-07-30   125225    2045
23907             Tennessee  2021-07-30   893554   12724
24463                 Texas  2021-07-30  3133639   53248
25019                  Utah  2021-07-30   325592    1670
25575               Vermont  2021-07-30    24889     260
26131              Virginia  2021-07-30   694384   11532
26687            Washington  2021-07-30   473076    6122
27243         West Virginia  2021-07-30   167016    2946
27799             Wisconsin  2021-07-30   687198    8290
28355               Wyoming  2021-07-30    65127     776

### Inspect the Vaccination Data

In [41]:
dfvac = pd.read_csv("data/us_state_vaccinations.csv")

In [42]:
dfvac.head()

date location  total_vaccinations  total_distributed  \
0  2021-01-12  Alabama             78134.0           377025.0   
1  2021-01-13  Alabama             84040.0           378975.0   
2  2021-01-14  Alabama             92300.0           435350.0   
3  2021-01-15  Alabama            100567.0           444650.0   
4  2021-01-16  Alabama                 NaN                NaN   

   people_vaccinated  people_fully_vaccinated_per_hundred  \
0            70861.0                                 0.15   
1            74792.0                                 0.19   
2            80480.0                                  NaN   
3            86956.0                                 0.28   
4                NaN                                  NaN   

   total_vaccinations_per_hundred  people_fully_vaccinated  \
0                            1.59                   7270.0   
1                            1.71                   9245.0   
2                            1.88                      NaN   
3                            2.05                  13488.0   
4                             NaN                      NaN   

   people_vaccinated_per_hundred  distributed_per_hundred  \
0                           1.45                     7.69   
1                           1.53                     7.73   
2                           1.64                     8.88   
3                           1.77                     9.07   
4                            NaN                      NaN   

   daily_vaccinations_raw  daily_vaccinations  daily_vaccinations_per_million  \
0                     NaN                 NaN                             NaN   
1                  5906.0              5906.0                          1205.0   
2                  8260.0              7083.0                          1445.0   
3                  8267.0              7478.0                          1525.0   
4                  7557.0              7498.0                          1529.0   

   share_doses_used  
0             0.207  
1             0.222  
2             0.212  
3             0.226  
4               NaN

In [43]:
print("Vaccine dataset has", dfvac.shape[0], "rows and", dfvac.shape[1], "columns in the dataset.")

Vaccine dataset has 13088 rows and 14 columns in the dataset.


Since the dataset contains the time series state level data of US vaccination, we subsetted the current date as an example to examine the data.

In [44]:
dfvac_current = dfvac[dfvac.date == '2021-07-30']
dfvac_current.head()

date        location  total_vaccinations  total_distributed  \
199   2021-07-30         Alabama           3586810.0          5092510.0   
400   2021-07-30          Alaska            696377.0           853805.0   
601   2021-07-30  American Samoa             51670.0            55830.0   
802   2021-07-30         Arizona           7085856.0          8540830.0   
1003  2021-07-30        Arkansas           2432736.0          3004150.0   

      people_vaccinated  people_fully_vaccinated_per_hundred  \
199           2116551.0                                34.33   
400            376154.0                                45.48   
601             28768.0                                42.31   
802           3853257.0                                45.19   
1003          1405459.0                                36.29   

      total_vaccinations_per_hundred  people_fully_vaccinated  \
199                            73.15                1683367.0   
400                            95.19                 332678.0   
601                            92.78                  23560.0   
802                            97.35                3289110.0   
1003                           80.61                1095214.0   

      people_vaccinated_per_hundred  distributed_per_hundred  \
199                           43.17                   103.86   
400                           51.42                   116.71   
601                           51.66                   100.25   
802                           52.94                   117.34   
1003                          46.57                    99.55   

      daily_vaccinations_raw  daily_vaccinations  \
199                  28844.0             13131.0   
400                   1214.0               837.0   
601                      0.0               229.0   
802                  17540.0             12063.0   
1003                 10935.0              9842.0   

      daily_vaccinations_per_million  share_doses_used  
199                           2678.0             0.704  
400                           1144.0             0.816  
601                           4112.0             0.925  
802                           1657.0             0.830  
1003                          3261.0             0.810

In [45]:
dfvac_current.isnull().sum()

date                                   0
location                               0
total_vaccinations                     0
total_distributed                      1
people_vaccinated                      1
people_fully_vaccinated_per_hundred    4
total_vaccinations_per_hundred         4
people_fully_vaccinated                1
people_vaccinated_per_hundred          4
distributed_per_hundred                4
daily_vaccinations_raw                 0
daily_vaccinations                     0
daily_vaccinations_per_million         4
share_doses_used                       1
dtype: int64

Looking at the unique locations, we will only be keeping the states and the capital city

In [46]:
dfvac_current.location.unique()

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'Bureau of Prisons', 'California', 'Colorado', 'Connecticut',
       'Delaware', 'Dept of Defense', 'District of Columbia',
       'Federated States of Micronesia', 'Florida', 'Georgia', 'Guam',
       'Hawaii', 'Idaho', 'Illinois', 'Indian Health Svc', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Long Term Care', 'Louisiana',
       'Maine', 'Marshall Islands', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York State', 'North Carolina', 'North Dakota',
       'Northern Mariana Islands', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Republic of Palau', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas',
       'United States', 'Utah', 'Vermont', 'Veterans Health',
       'Virgin Islands', 'Virginia', 'Washington', 'West V

Only location where it is a state will be kept, and also we need to change New York State to New York in order to match with other datasets. 

In [47]:
dfvac_current = dfvac_current[~dfvac_current.location.isin(['American Samoa','Bureau of Prisons','Dept of Defense','Federated States of Micronesia',
               'Indian Health Svc','Long Term Care','Marshall Islands','Northern Mariana Islands',
               'Puerto Rico', 'Republic of Palau','United States','Veterans Health','Guam','Virgin Islands'])]
dfvac_current.location[dfvac_current.location == "New York State"] = "New York"
dfvac_current.location.nunique()

51

Similarly, we removed those locations from the time series dataframe as well. 

In [48]:
dfvac = dfvac[~dfvac.location.isin(['American Samoa','Bureau of Prisons','Dept of Defense','Federated States of Micronesia',
               'Indian Health Svc','Long Term Care','Marshall Islands','Northern Mariana Islands',
               'Puerto Rico', 'Republic of Palau','United States','Veterans Health','Guam','Virgin Islands'])]
dfvac.location[dfvac.location == "New York State"] = "New York"
dfvac.location.nunique()

51

In [49]:
dfvac[dfvac.location == "New York"]

date  location  total_vaccinations  total_distributed  \
8040  2021-01-12  New York            579532.0          1622100.0   
8041  2021-01-13  New York            632473.0          1796850.0   
8042  2021-01-14  New York            688576.0          1872625.0   
8043  2021-01-15  New York            757466.0          1884325.0   
8044  2021-01-16  New York                 NaN                NaN   
...          ...       ...                 ...                ...   
8236  2021-07-27  New York          22775948.0         24823865.0   
8237  2021-07-28  New York          22820585.0         24861515.0   
8238  2021-07-29  New York          22860905.0         24980615.0   
8239  2021-07-30  New York          22902722.0         25076125.0   
8240  2021-07-31  New York          22950250.0         25122005.0   

      people_vaccinated  people_fully_vaccinated_per_hundred  \
8040           543014.0                                 0.19   
8041           587865.0                                 0.23   
8042           617596.0                                  NaN   
8043           671311.0                                 0.38   
8044                NaN                                  NaN   
...                 ...                                  ...   
8236         12168754.0                                56.72   
8237         12191988.0                                56.83   
8238         12216355.0                                56.92   
8239         12242044.0                                57.01   
8240         12271911.0                                57.11   

      total_vaccinations_per_hundred  people_fully_vaccinated  \
8040                            2.98                  36422.0   
8041                            3.25                  44512.0   
8042                            3.54                      NaN   
8043                            3.89                  73523.0   
8044                             NaN                      NaN   
...                              ...                      ...   
8236                          117.08               11033818.0   
8237                          117.31               11056206.0   
8238                          117.52               11073293.0   
8239                          117.73               11091103.0   
8240                          117.97               11109858.0   

      people_vaccinated_per_hundred  distributed_per_hundred  \
8040                           2.79                     8.34   
8041                           3.02                     9.24   
8042                           3.17                     9.63   
8043                           3.45                     9.69   
8044                            NaN                      NaN   
...                             ...                      ...   
8236                          62.55                   127.61   
8237                          62.67                   127.80   
8238                          62.80                   128.41   
8239                          62.93                   128.90   
8240                          63.08                   129.14   

      daily_vaccinations_raw  daily_vaccinations  \
8040                     NaN                 NaN   
8041                52941.00             52941.0   
8042                56103.00             54522.0   
8043                68890.00             59311.0   
8044                60713.25             59662.0   
...                      ...                 ...   
8236                43644.00             36471.0   
8237                44637.00             37570.0   
8238                40320.00             37158.0   
8239                41817.00             37673.0   
8240                47528.00             37731.0   

      daily_vaccinations_per_million  share_doses_used  
8040                             NaN             0.357  
8041                          2721.0             0.352  
8042                          2803.0             0.368  
8043       

checking for missing values again. We do not have missing values anymore.

In [50]:
dfvac_current.isnull().sum()

date                                   0
location                               0
total_vaccinations                     0
total_distributed                      0
people_vaccinated                      0
people_fully_vaccinated_per_hundred    0
total_vaccinations_per_hundred         0
people_fully_vaccinated                0
people_vaccinated_per_hundred          0
distributed_per_hundred                0
daily_vaccinations_raw                 0
daily_vaccinations                     0
daily_vaccinations_per_million         0
share_doses_used                       0
dtype: int64

now we have the 50 states and the capital city in the data

In [51]:
print("Now, the number of rows in the dataset is equal to the number of unique values in column 'location':", 
      len(dfvac_current) == dfvac_current.location.nunique())

Now, the number of rows in the dataset is equal to the number of unique values in column 'location': True


Since the number of rows of the dataframe is equal to the number of unique values for variable "location". We can consider the column "location" as the key column of the dataframe. That is, each row in dataframe current represents a different US county.

In [52]:
dfvac = dfvac[['date', 'location', 'people_vaccinated', 'people_fully_vaccinated_per_hundred',
       'total_vaccinations_per_hundred', 'people_fully_vaccinated',
       'people_vaccinated_per_hundred', 'daily_vaccinations',
       'daily_vaccinations_per_million']]

In [53]:
dfvac_current = dfvac_current[['date', 'location', 'people_vaccinated', 'people_fully_vaccinated_per_hundred',
       'total_vaccinations_per_hundred', 'people_fully_vaccinated',
       'people_vaccinated_per_hundred', 'daily_vaccinations',
       'daily_vaccinations_per_million']]

## Merge county-level data

#### Merge the current county-level data
Merge County info with covid cases by county (current only)

In [54]:
mergeCounty = pd.merge(dfCounty, dfcovCnty_current, how='inner', on='fips', validate = "1:1")
mergeCounty.head(5)

fips        county_x  state_x state_code_x    male  female  median_age  \
0  1001  Autauga County  Alabama           AL   26874   28326        37.8   
1  1003  Baldwin County  Alabama           AL  101188  106919        42.8   
2  1005  Barbour County  Alabama           AL   13697   12085        39.9   
3  1007     Bibb County  Alabama           AL   12152   10375        39.9   
4  1009   Blount County  Alabama           AL   28434   29211        40.8   

   population  female_percentage      lat_x     long_x county_y  state_y  \
0       55200          51.315217  32.534923 -86.642730  Autauga  Alabama   
1      208107          51.376936  30.727479 -87.722564  Baldwin  Alabama   
2       25782          46.873788  31.869581 -85.393210  Barbour  Alabama   
3       22527          46.055844  32.998628 -87.126475     Bibb  Alabama   
4       57645          50.673953  33.980869 -86.567380   Blount  Alabama   

       lat_y     long_y        date  cases state_code_y  deaths  
0  32.539527 -86.644082  2021-07-30   7479           AL     114  
1  30.727750 -87.722071  2021-07-30  24499           AL     328  
2  31.868263 -85.387129  2021-07-30   2462           AL      61  
3  32.996421 -87.125115  2021-07-30   2845           AL      65  
4  33.982109 -86.567906  2021-07-30   7266           AL     139

Merge the above with poverty data

In [55]:
mergeCouPov = pd.merge(mergeCounty, dfPoverty, how='inner', left_on='fips', right_on='FIPStxt', validate = "1:1")
mergeCouPov.head(5)

fips        county_x  state_x state_code_x    male  female  median_age  \
0  1001  Autauga County  Alabama           AL   26874   28326        37.8   
1  1003  Baldwin County  Alabama           AL  101188  106919        42.8   
2  1005  Barbour County  Alabama           AL   13697   12085        39.9   
3  1007     Bibb County  Alabama           AL   12152   10375        39.9   
4  1009   Blount County  Alabama           AL   28434   29211        40.8   

   population  female_percentage      lat_x  ...  FIPStxt Stabr  \
0       55200          51.315217  32.534923  ...     1001    AL   
1      208107          51.376936  30.727479  ...     1003    AL   
2       25782          46.873788  31.869581  ...     1005    AL   
3       22527          46.055844  32.998628  ...     1007    AL   
4       57645          50.673953  33.980869  ...     1009    AL   

        Area_name  Rural-urban_Continuum_Code_2013  \
0  Autauga County                          Level 2   
1  Baldwin County                          Level 3   
2  Barbour County                          Level 6   
3     Bibb County                          Level 1   
4   Blount County                          Level 1   

                           Urban_Influence_Code_2013 POVALL_2018  \
0  In small metro area of less than 1 million res...        7587   
1  In small metro area of less than 1 million res...       21069   
2  Noncore adjacent to small metro area and conta...        6788   
3        In large metro area of 1+ million residents        4400   
4        In large metro area of 1+ million residents        7527   

   PCTPOVALL_2018 POV017_2018  PCTPOV017_2018  MEDHHINC_2018  
0            13.8        2509            19.3          59338  
1             9.8        6442            13.9          57588  
2            30.9        2242            43.9          34382  
3            21.8        1238            27.8          46064  
4            13.2        2374            18.0          50412  

[5 rows x 29 columns]

In [56]:
mergeCouPov.isnull().sum()

fips                               0
county_x                           0
state_x                            0
state_code_x                       1
male                               0
female                             0
median_age                         0
population                         0
female_percentage                  0
lat_x                              0
long_x                             0
county_y                           0
state_y                            0
lat_y                              0
long_y                             0
date                               0
cases                              0
state_code_y                       1
deaths                             0
FIPStxt                            0
Stabr                              0
Area_name                          0
Rural-urban_Continuum_Code_2013    0
Urban_Influence_Code_2013          0
POVALL_2018                        0
PCTPOVALL_2018                     0
POV017_2018                        0
P

Then we checked the missing values and found out that the missing values occur due to the fact that District of Columbia does not have a state code. 

In [57]:
mergeCouPov[mergeCouPov.state_code_x.isnull()]

fips              county_x               state_x state_code_x    male  \
319  11001  District of Columbia  District of Columbia          NaN  324881   

     female  median_age  population  female_percentage      lat_x  ...  \
319  359617        33.9      684498          52.537334  38.904734  ...   

     FIPStxt Stabr             Area_name  Rural-urban_Continuum_Code_2013  \
319    11001    DC  District of Columbia                          Level 1   

                       Urban_Influence_Code_2013 POVALL_2018  PCTPOVALL_2018  \
319  In large metro area of 1+ million residents      107806            16.1   

    POV017_2018  PCTPOV017_2018  MEDHHINC_2018  
319       30803            24.5          82533  

[1 rows x 29 columns]

In [58]:
mergeCouPov[mergeCouPov.state_code_y.isnull()]

fips              county_x               state_x state_code_x    male  \
319  11001  District of Columbia  District of Columbia          NaN  324881   

     female  median_age  population  female_percentage      lat_x  ...  \
319  359617        33.9      684498          52.537334  38.904734  ...   

     FIPStxt Stabr             Area_name  Rural-urban_Continuum_Code_2013  \
319    11001    DC  District of Columbia                          Level 1   

                       Urban_Influence_Code_2013 POVALL_2018  PCTPOVALL_2018  \
319  In large metro area of 1+ million residents      107806            16.1   

    POV017_2018  PCTPOV017_2018  MEDHHINC_2018  
319       30803            24.5          82533  

[1 rows x 29 columns]

Then, we removed some of the redundant columns and kept the rest.

In [59]:
county_current = mergeCouPov[['fips', 'county_x', 'state_x', 'state_code_x', 'male', 'female',
       'median_age', 'population', 'female_percentage', 'lat_x', 'long_x', 'date', 'cases',
       'deaths', 'Rural-urban_Continuum_Code_2013', 'Urban_Influence_Code_2013',
       'POVALL_2018', 'PCTPOVALL_2018', 'POV017_2018', 'PCTPOV017_2018',
       'MEDHHINC_2018']]

In [60]:
county_current.head(5)

fips        county_x  state_x state_code_x    male  female  median_age  \
0  1001  Autauga County  Alabama           AL   26874   28326        37.8   
1  1003  Baldwin County  Alabama           AL  101188  106919        42.8   
2  1005  Barbour County  Alabama           AL   13697   12085        39.9   
3  1007     Bibb County  Alabama           AL   12152   10375        39.9   
4  1009   Blount County  Alabama           AL   28434   29211        40.8   

   population  female_percentage      lat_x  ...        date  cases  deaths  \
0       55200          51.315217  32.534923  ...  2021-07-30   7479     114   
1      208107          51.376936  30.727479  ...  2021-07-30  24499     328   
2       25782          46.873788  31.869581  ...  2021-07-30   2462      61   
3       22527          46.055844  32.998628  ...  2021-07-30   2845      65   
4       57645          50.673953  33.980869  ...  2021-07-30   7266     139   

   Rural-urban_Continuum_Code_2013  \
0                          Level 2   
1                          Level 3   
2                          Level 6   
3                          Level 1   
4                          Level 1   

                           Urban_Influence_Code_2013 POVALL_2018  \
0  In small metro area of less than 1 million res...        7587   
1  In small metro area of less than 1 million res...       21069   
2  Noncore adjacent to small metro area and conta...        6788   
3        In large metro area of 1+ million residents        4400   
4        In large metro area of 1+ million residents        7527   

   PCTPOVALL_2018  POV017_2018  PCTPOV017_2018  MEDHHINC_2018  
0            13.8         2509            19.3          59338  
1             9.8         6442            13.9          57588  
2            30.9         2242            43.9          34382  
3            21.8         1238            27.8          46064  
4            13.2         2374            18.0          50412  

[5 rows x 21 columns]

#### Merge time series county-level data

In [61]:
mergeCountyts = pd.merge(dfCounty, dfcovCnty, how='inner', on='fips', validate = "1:m")
mergeCountyts.head(5)

fips        county_x  state_x state_code_x   male  female  median_age  \
0  1001  Autauga County  Alabama           AL  26874   28326        37.8   
1  1001  Autauga County  Alabama           AL  26874   28326        37.8   
2  1001  Autauga County  Alabama           AL  26874   28326        37.8   
3  1001  Autauga County  Alabama           AL  26874   28326        37.8   
4  1001  Autauga County  Alabama           AL  26874   28326        37.8   

   population  female_percentage      lat_x    long_x county_y  state_y  \
0       55200          51.315217  32.534923 -86.64273  Autauga  Alabama   
1       55200          51.315217  32.534923 -86.64273  Autauga  Alabama   
2       55200          51.315217  32.534923 -86.64273  Autauga  Alabama   
3       55200          51.315217  32.534923 -86.64273  Autauga  Alabama   
4       55200          51.315217  32.534923 -86.64273  Autauga  Alabama   

       lat_y     long_y        date  cases state_code_y  deaths  
0  32.539527 -86.644082  2020-01-22      0           AL       0  
1  32.539527 -86.644082  2020-01-23      0           AL       0  
2  32.539527 -86.644082  2020-01-24      0           AL       0  
3  32.539527 -86.644082  2020-01-25      0           AL       0  
4  32.539527 -86.644082  2020-01-26      0           AL       0

In [62]:
mergeCouPovts = pd.merge(mergeCountyts, dfPoverty, how='inner', left_on='fips', right_on='FIPStxt', validate = "m:1")
mergeCouPovts.head(5)

fips        county_x  state_x state_code_x   male  female  median_age  \
0  1001  Autauga County  Alabama           AL  26874   28326        37.8   
1  1001  Autauga County  Alabama           AL  26874   28326        37.8   
2  1001  Autauga County  Alabama           AL  26874   28326        37.8   
3  1001  Autauga County  Alabama           AL  26874   28326        37.8   
4  1001  Autauga County  Alabama           AL  26874   28326        37.8   

   population  female_percentage      lat_x  ...  FIPStxt Stabr  \
0       55200          51.315217  32.534923  ...     1001    AL   
1       55200          51.315217  32.534923  ...     1001    AL   
2       55200          51.315217  32.534923  ...     1001    AL   
3       55200          51.315217  32.534923  ...     1001    AL   
4       55200          51.315217  32.534923  ...     1001    AL   

        Area_name  Rural-urban_Continuum_Code_2013  \
0  Autauga County                          Level 2   
1  Autauga County                          Level 2   
2  Autauga County                          Level 2   
3  Autauga County                          Level 2   
4  Autauga County                          Level 2   

                           Urban_Influence_Code_2013 POVALL_2018  \
0  In small metro area of less than 1 million res...        7587   
1  In small metro area of less than 1 million res...        7587   
2  In small metro area of less than 1 million res...        7587   
3  In small metro area of less than 1 million res...        7587   
4  In small metro area of less than 1 million res...        7587   

   PCTPOVALL_2018 POV017_2018  PCTPOV017_2018  MEDHHINC_2018  
0            13.8        2509            19.3          59338  
1            13.8        2509            19.3          59338  
2            13.8        2509            19.3          59338  
3            13.8        2509            19.3          59338  
4            13.8        2509            19.3          59338  

[5 rows x 29 columns]

In [63]:
mergeCouPovts.isnull().sum()

fips                                 0
county_x                             0
state_x                              0
state_code_x                       556
male                                 0
female                               0
median_age                           0
population                           0
female_percentage                    0
lat_x                                0
long_x                               0
county_y                             0
state_y                              0
lat_y                                0
long_y                               0
date                                 0
cases                                0
state_code_y                       556
deaths                               0
FIPStxt                              0
Stabr                                0
Area_name                            0
Rural-urban_Continuum_Code_2013      0
Urban_Influence_Code_2013            0
POVALL_2018                          0
PCTPOVALL_2018           

In [64]:
mergeCouPovts[mergeCouPovts.state_code_x.isnull()].county_x.unique()

array(['District of Columbia'], dtype=object)

In [65]:
mergeCouPovts[mergeCouPovts.state_code_y.isnull()].county_x.unique()

array(['District of Columbia'], dtype=object)

Similarly, the missing values here are also due to District of Columbia not having a state code. 

In [66]:
county_ts = mergeCouPovts[['fips', 'county_x', 'state_x', 'state_code_x', 'male', 'female',
       'median_age', 'population', 'female_percentage', 'lat_x', 'long_x', 'date', 'cases',
       'deaths', 'Rural-urban_Continuum_Code_2013', 'Urban_Influence_Code_2013',
       'POVALL_2018', 'PCTPOVALL_2018', 'POV017_2018', 'PCTPOV017_2018',
       'MEDHHINC_2018']]

Also, we removed the redundant columns like the current data. 

In [67]:
county_ts.head()

fips        county_x  state_x state_code_x   male  female  median_age  \
0  1001  Autauga County  Alabama           AL  26874   28326        37.8   
1  1001  Autauga County  Alabama           AL  26874   28326        37.8   
2  1001  Autauga County  Alabama           AL  26874   28326        37.8   
3  1001  Autauga County  Alabama           AL  26874   28326        37.8   
4  1001  Autauga County  Alabama           AL  26874   28326        37.8   

   population  female_percentage      lat_x  ...        date cases  deaths  \
0       55200          51.315217  32.534923  ...  2020-01-22     0       0   
1       55200          51.315217  32.534923  ...  2020-01-23     0       0   
2       55200          51.315217  32.534923  ...  2020-01-24     0       0   
3       55200          51.315217  32.534923  ...  2020-01-25     0       0   
4       55200          51.315217  32.534923  ...  2020-01-26     0       0   

   Rural-urban_Continuum_Code_2013  \
0                          Level 2   
1                          Level 2   
2                          Level 2   
3                          Level 2   
4                          Level 2   

                           Urban_Influence_Code_2013 POVALL_2018  \
0  In small metro area of less than 1 million res...        7587   
1  In small metro area of less than 1 million res...        7587   
2  In small metro area of less than 1 million res...        7587   
3  In small metro area of less than 1 million res...        7587   
4  In small metro area of less than 1 million res...        7587   

   PCTPOVALL_2018  POV017_2018  PCTPOV017_2018  MEDHHINC_2018  
0            13.8         2509            19.3          59338  
1            13.8         2509            19.3          59338  
2            13.8         2509            19.3          59338  
3            13.8         2509            19.3          59338  
4            13.8         2509            19.3          59338  

[5 rows x 21 columns]

## Merge state-level data

#### Merge current state-level data

In [68]:
mergeCovState = pd.merge(dfcovSt_current, state_pop, how='inner', on = 'state')
mergeCovState

state        date    cases  deaths      male    female  \
0                Alabama  2021-07-30   577463   11510   2355799   2508881   
1                 Alaska  2021-07-30    75486     389    385579    352937   
2                Arizona  2021-07-30   925169   18224   3453439   3493246   
3               Arkansas  2021-07-30   385113    6123   1468412   1522259   
4             California  2021-07-30  3956303   64423  19453769  19694991   
5               Colorado  2021-07-30   575082    6945   2780831   2750310   
6            Connecticut  2021-07-30   354335    8293   1747131   1834373   
7               Delaware  2021-07-30   111275    1830    459570    489925   
8   District of Columbia  2021-07-30    50398    1149    324881    359617   
9                Florida  2021-07-30  2634234   39079  10071925  10526214   
10               Georgia  2021-07-30  1179449   21676   5012229   5285255   
11                Hawaii  2021-07-30    41925     537    713772    708257   
12                 Idaho  2021-07-30   200484    2197    845564    842245   
13              Illinois  2021-07-30  1419611   25873   6295915   6525582   
14               Indiana  2021-07-30   771299   14005   3271068   3366358   
15                  Iowa  2021-07-30   378831    6183   1554481   1578018   
16                Kansas  2021-07-30   333850    5261   1449413   1459363   
17              Kentucky  2021-07-30   482599    7334   2186715   2253489   
18             Louisiana  2021-07-30   541679   10999   2279968   2383648   
19                 Maine  2021-07-30    70372     899    652299    680514   
20              Maryland  2021-07-30   467961    9818   2910323   3093112   
21         Massachusetts  2021-07-30   716771   18076   3313979   3516214   
22              Michigan  2021-07-30   984016   21028   4899118   5058370   
23             Minnesota  2021-07-30   612701    7761   2751748   2775610   
24           Mississippi  2021-07-30   343505    7543   1450379   1538383   
25              Missouri  2021-07-30   634482    9464   2988772   3101290   
26               Montana  2021-07-30   116366    1704    524220    517512   
27              Nebraska  2021-07-30   228450    2280    949910    954850   
28                Nevada  2021-07-30   356401    5912   1467165   1455684   
29         New Hampshire  2021-07-30   100657    1387    664983    678639   
30            New Jersey  2021-07-30  1038171   26595   4335930   4545915   
31            New Mexico  2021-07-30   210416    4410   1035850   1056584   
32              New York  2021-07-30  2147318   53632   9519555  10098898   
33        North Carolina  2021-07-30  1048076   13635   4944759   5210865   
34          North Dakota  2021-07-30   111570    1570    386183    366018   
35                  Ohio  2021-07-30  1127808   20492   5703967   5937912   
36              Oklahoma  2021-07-30   480635    7485   1941576   1976561   
37                Oregon  2021-07-30   219755    2858   2023021   2058922   
38          Pennsylvania  2021-07-30  1229002   27850   6263189   6527992   
39          Rhode Island  2021-07-30   154339    2740    513332    543279   
40        South Carolina  2021-07-30   617148    9904   2405726   2550199   
41          South Dakota  2021-07-30   125225    2045    435761    428528   
42             Tennessee  2021-07-30   893554   12724   3243678   3407411   
43                 Texas  2021-07-30  3133639   53248  13849775  14035420   
44                  Utah  2021-07-30   325592    1670   1532768   1512582   
45               Vermont  2021-07-30    24889     260    307973    317004   
46              Virginia  2021-07-30   694384   11532   4138574   4275200   
47            Washington  2021-07-30   473076    6122   3645665   3648671   
48         West Virginia  2021-07-30   167016    2946    904196    924858   
49             Wisconsin  2021-07-30   687198    8290   2872376   2906018   
50               Wyoming  2021-07-30    65127     776    296979    284857   

    population

In [69]:
mergeCovStatePov = pd.merge(mergeCovState, dfPoverty, how = "inner", left_on = "state", right_on = "Area_name")
mergeCovStatePov

state        date    cases  deaths      male    female  \
0                Alabama  2021-07-30   577463   11510   2355799   2508881   
1                 Alaska  2021-07-30    75486     389    385579    352937   
2                Arizona  2021-07-30   925169   18224   3453439   3493246   
3               Arkansas  2021-07-30   385113    6123   1468412   1522259   
4             California  2021-07-30  3956303   64423  19453769  19694991   
5               Colorado  2021-07-30   575082    6945   2780831   2750310   
6            Connecticut  2021-07-30   354335    8293   1747131   1834373   
7               Delaware  2021-07-30   111275    1830    459570    489925   
8   District of Columbia  2021-07-30    50398    1149    324881    359617   
9   District of Columbia  2021-07-30    50398    1149    324881    359617   
10               Florida  2021-07-30  2634234   39079  10071925  10526214   
11               Georgia  2021-07-30  1179449   21676   5012229   5285255   
12                Hawaii  2021-07-30    41925     537    713772    708257   
13                 Idaho  2021-07-30   200484    2197    845564    842245   
14              Illinois  2021-07-30  1419611   25873   6295915   6525582   
15               Indiana  2021-07-30   771299   14005   3271068   3366358   
16                  Iowa  2021-07-30   378831    6183   1554481   1578018   
17                Kansas  2021-07-30   333850    5261   1449413   1459363   
18              Kentucky  2021-07-30   482599    7334   2186715   2253489   
19             Louisiana  2021-07-30   541679   10999   2279968   2383648   
20                 Maine  2021-07-30    70372     899    652299    680514   
21              Maryland  2021-07-30   467961    9818   2910323   3093112   
22         Massachusetts  2021-07-30   716771   18076   3313979   3516214   
23              Michigan  2021-07-30   984016   21028   4899118   5058370   
24             Minnesota  2021-07-30   612701    7761   2751748   2775610   
25           Mississippi  2021-07-30   343505    7543   1450379   1538383   
26              Missouri  2021-07-30   634482    9464   2988772   3101290   
27               Montana  2021-07-30   116366    1704    524220    517512   
28              Nebraska  2021-07-30   228450    2280    949910    954850   
29                Nevada  2021-07-30   356401    5912   1467165   1455684   
30         New Hampshire  2021-07-30   100657    1387    664983    678639   
31            New Jersey  2021-07-30  1038171   26595   4335930   4545915   
32            New Mexico  2021-07-30   210416    4410   1035850   1056584   
33              New York  2021-07-30  2147318   53632   9519555  10098898   
34        North Carolina  2021-07-30  1048076   13635   4944759   5210865   
35          North Dakota  2021-07-30   111570    1570    386183    366018   
36                  Ohio  2021-07-30  1127808   20492   5703967   5937912   
37              Oklahoma  2021-07-30   480635    7485   1941576   1976561   
38                Oregon  2021-07-30   219755    2858   2023021   2058922   
39          Pennsylvania  2021-07-30  1229002   27850   6263189   6527992   
40          Rhode Island  2021-07-30   154339    2740    513332    543279   
41        South Carolina  2021-07-30   617148    9904   2405726   2550199   
42          South Dakota  2021-07-30   125225    2045    435761    428528   
43             Tennessee  2021-07-30   893554   12724   3243678   3407411   
44                 Texas  2021-07-30  3133639   53248  13849775  14035420   
45                  Utah  2021-07-30   325592    1670   1532768   1512582   
46               Vermont  2021-07-30    24889     260    307973    317004   
47              Virginia  2021-07-30   694384   11532   4138574   4275200   
48            Washington  2021-07-30   473076    6122   3645665   3648671   
49         West Virginia  2021-07-30   167016    2946    904196    924858   
50             Wisconsin  2021-07-30   687198    8290   2872376   2906018   
51             

We found out that there are two observations of District of Columbia, thus we removed the second one. 

In [70]:
mergeCovStatePov.drop(mergeCovStatePov[mergeCovStatePov.FIPStxt == 11001].index, inplace = True)
len(mergeCovStatePov)

51

In [71]:
mergeCovStatePovVac = pd.merge(mergeCovStatePov, dfvac_current, how = "inner", left_on = "state", right_on = "location")
mergeCovStatePovVac

state      date_x    cases  deaths      male    female  \
0                Alabama  2021-07-30   577463   11510   2355799   2508881   
1                 Alaska  2021-07-30    75486     389    385579    352937   
2                Arizona  2021-07-30   925169   18224   3453439   3493246   
3               Arkansas  2021-07-30   385113    6123   1468412   1522259   
4             California  2021-07-30  3956303   64423  19453769  19694991   
5               Colorado  2021-07-30   575082    6945   2780831   2750310   
6            Connecticut  2021-07-30   354335    8293   1747131   1834373   
7               Delaware  2021-07-30   111275    1830    459570    489925   
8   District of Columbia  2021-07-30    50398    1149    324881    359617   
9                Florida  2021-07-30  2634234   39079  10071925  10526214   
10               Georgia  2021-07-30  1179449   21676   5012229   5285255   
11                Hawaii  2021-07-30    41925     537    713772    708257   
12                 Idaho  2021-07-30   200484    2197    845564    842245   
13              Illinois  2021-07-30  1419611   25873   6295915   6525582   
14               Indiana  2021-07-30   771299   14005   3271068   3366358   
15                  Iowa  2021-07-30   378831    6183   1554481   1578018   
16                Kansas  2021-07-30   333850    5261   1449413   1459363   
17              Kentucky  2021-07-30   482599    7334   2186715   2253489   
18             Louisiana  2021-07-30   541679   10999   2279968   2383648   
19                 Maine  2021-07-30    70372     899    652299    680514   
20              Maryland  2021-07-30   467961    9818   2910323   3093112   
21         Massachusetts  2021-07-30   716771   18076   3313979   3516214   
22              Michigan  2021-07-30   984016   21028   4899118   5058370   
23             Minnesota  2021-07-30   612701    7761   2751748   2775610   
24           Mississippi  2021-07-30   343505    7543   1450379   1538383   
25              Missouri  2021-07-30   634482    9464   2988772   3101290   
26               Montana  2021-07-30   116366    1704    524220    517512   
27              Nebraska  2021-07-30   228450    2280    949910    954850   
28                Nevada  2021-07-30   356401    5912   1467165   1455684   
29         New Hampshire  2021-07-30   100657    1387    664983    678639   
30            New Jersey  2021-07-30  1038171   26595   4335930   4545915   
31            New Mexico  2021-07-30   210416    4410   1035850   1056584   
32              New York  2021-07-30  2147318   53632   9519555  10098898   
33        North Carolina  2021-07-30  1048076   13635   4944759   5210865   
34          North Dakota  2021-07-30   111570    1570    386183    366018   
35                  Ohio  2021-07-30  1127808   20492   5703967   5937912   
36              Oklahoma  2021-07-30   480635    7485   1941576   1976561   
37                Oregon  2021-07-30   219755    2858   2023021   2058922   
38          Pennsylvania  2021-07-30  1229002   27850   6263189   6527992   
39          Rhode Island  2021-07-30   154339    2740    513332    543279   
40        South Carolina  2021-07-30   617148    9904   2405726   2550199   
41          South Dakota  2021-07-30   125225    2045    435761    428528   
42             Tennessee  2021-07-30   893554   12724   3243678   3407411   
43                 Texas  2021-07-30  3133639   53248  13849775  14035420   
44                  Utah  2021-07-30   325592    1670   1532768   1512582   
45               Vermont  2021-07-30    24889     260    307973    317004   
46              Virginia  2021-07-30   694384   11532   4138574   4275200   
47            Washington  2021-07-30   473076    6122   3645665   3648671   
48         West Virginia  2021-07-30   167016    2946    904196    924858   
49             Wisconsin  2021-07-30   687198    8290   2872376   2906018   
50               Wyoming  2021-07-30    65127     776    296979    284857   

    population

In [72]:
mergeCovStatePovVac.columns

Index(['state', 'date_x', 'cases', 'deaths', 'male', 'female', 'population',
       'female_proportion', 'FIPStxt', 'Stabr', 'Area_name',
       'Rural-urban_Continuum_Code_2013', 'Urban_Influence_Code_2013',
       'POVALL_2018', 'PCTPOVALL_2018', 'POV017_2018', 'PCTPOV017_2018',
       'MEDHHINC_2018', 'date_y', 'location', 'people_vaccinated',
       'people_fully_vaccinated_per_hundred', 'total_vaccinations_per_hundred',
       'people_fully_vaccinated', 'people_vaccinated_per_hundred',
       'daily_vaccinations', 'daily_vaccinations_per_million'],
      dtype='object')

In [73]:
state_current = mergeCovStatePovVac[['state', 'date_x', 'cases', 'deaths', 'male', 'female', 'population',
       'female_proportion', 'FIPStxt', 'Stabr', 
       'POVALL_2018', 'PCTPOVALL_2018', 'POV017_2018', 'PCTPOV017_2018',
       'MEDHHINC_2018', 'people_vaccinated', 'people_fully_vaccinated_per_hundred', 'total_vaccinations_per_hundred',
       'people_fully_vaccinated', 'people_vaccinated_per_hundred',
       'daily_vaccinations', 'daily_vaccinations_per_million']]

### Merge time series state-level data

In [89]:
mergeCovStatets = pd.merge(dfcovSt, state_pop, how='inner', on = 'state')
mergeCovStatets.head()

state        date  cases  deaths     male   female  population  \
0  Alabama  2020-01-22      0       0  2355799  2508881     4864680   
1  Alabama  2020-01-23      0       0  2355799  2508881     4864680   
2  Alabama  2020-01-24      0       0  2355799  2508881     4864680   
3  Alabama  2020-01-25      0       0  2355799  2508881     4864680   
4  Alabama  2020-01-26      0       0  2355799  2508881     4864680   

   female_proportion  
0           0.515734  
1           0.515734  
2           0.515734  
3           0.515734  
4           0.515734

In [88]:
mergeCovStatePovts = pd.merge(mergeCovStatets, dfPoverty, how = "inner", left_on = "state", right_on = "Area_name")
mergeCovStatePovts.head()

state        date  cases  deaths     male   female  population  \
0  Alabama  2020-01-22      0       0  2355799  2508881     4864680   
1  Alabama  2020-01-23      0       0  2355799  2508881     4864680   
2  Alabama  2020-01-24      0       0  2355799  2508881     4864680   
3  Alabama  2020-01-25      0       0  2355799  2508881     4864680   
4  Alabama  2020-01-26      0       0  2355799  2508881     4864680   

   female_proportion  FIPStxt Stabr Area_name Rural-urban_Continuum_Code_2013  \
0           0.515734     1000    AL   Alabama                             NaN   
1           0.515734     1000    AL   Alabama                             NaN   
2           0.515734     1000    AL   Alabama                             NaN   
3           0.515734     1000    AL   Alabama                             NaN   
4           0.515734     1000    AL   Alabama                             NaN   

  Urban_Influence_Code_2013  POVALL_2018  PCTPOVALL_2018  POV017_2018  \
0                       NaN       801758            16.8       255613   
1                       NaN       801758            16.8       255613   
2                       NaN       801758            16.8       255613   
3                       NaN       801758            16.8       255613   
4                       NaN       801758            16.8       255613   

   PCTPOV017_2018  MEDHHINC_2018  
0            23.9          49881  
1            23.9          49881  
2            23.9          49881  
3            23.9          49881  
4            23.9          49881

In [87]:
mergeCovStatePovts.drop(mergeCovStatePovts[mergeCovStatePovts.FIPStxt == 11001].index, inplace = True)
mergeCovStatePovts.head()

state        date  cases  deaths     male   female  population  \
0  Alabama  2020-01-22      0       0  2355799  2508881     4864680   
1  Alabama  2020-01-23      0       0  2355799  2508881     4864680   
2  Alabama  2020-01-24      0       0  2355799  2508881     4864680   
3  Alabama  2020-01-25      0       0  2355799  2508881     4864680   
4  Alabama  2020-01-26      0       0  2355799  2508881     4864680   

   female_proportion  FIPStxt Stabr Area_name Rural-urban_Continuum_Code_2013  \
0           0.515734     1000    AL   Alabama                             NaN   
1           0.515734     1000    AL   Alabama                             NaN   
2           0.515734     1000    AL   Alabama                             NaN   
3           0.515734     1000    AL   Alabama                             NaN   
4           0.515734     1000    AL   Alabama                             NaN   

  Urban_Influence_Code_2013  POVALL_2018  PCTPOVALL_2018  POV017_2018  \
0                       NaN       801758            16.8       255613   
1                       NaN       801758            16.8       255613   
2                       NaN       801758            16.8       255613   
3                       NaN       801758            16.8       255613   
4                       NaN       801758            16.8       255613   

   PCTPOV017_2018  MEDHHINC_2018  
0            23.9          49881  
1            23.9          49881  
2            23.9          49881  
3            23.9          49881  
4            23.9          49881

In [86]:
mergeCovStatePovVacts = pd.merge(mergeCovStatePovts, dfvac, how = "inner", left_on = ["state", "date"], right_on = ["location", "date"])
mergeCovStatePovVacts.head()

state        date   cases  deaths     male   female  population  \
0  Alabama  2021-01-12  407848    5573  2355799  2508881     4864680   
1  Alabama  2021-01-13  410995    5760  2355799  2508881     4864680   
2  Alabama  2021-01-14  414583    5945  2355799  2508881     4864680   
3  Alabama  2021-01-15  417528    6030  2355799  2508881     4864680   
4  Alabama  2021-01-16  420681    6119  2355799  2508881     4864680   

   female_proportion  FIPStxt Stabr  ... PCTPOV017_2018 MEDHHINC_2018  \
0           0.515734     1000    AL  ...           23.9         49881   
1           0.515734     1000    AL  ...           23.9         49881   
2           0.515734     1000    AL  ...           23.9         49881   
3           0.515734     1000    AL  ...           23.9         49881   
4           0.515734     1000    AL  ...           23.9         49881   

  location  people_vaccinated  people_fully_vaccinated_per_hundred  \
0  Alabama            70861.0                                 0.15   
1  Alabama            74792.0                                 0.19   
2  Alabama            80480.0                                  NaN   
3  Alabama            86956.0                                 0.28   
4  Alabama                NaN                                  NaN   

   total_vaccinations_per_hundred  people_fully_vaccinated  \
0                            1.59                   7270.0   
1                            1.71                   9245.0   
2                            1.88                      NaN   
3                            2.05                  13488.0   
4                             NaN                      NaN   

   people_vaccinated_per_hundred daily_vaccinations  \
0                           1.45                NaN   
1                           1.53             5906.0   
2                           1.64             7083.0   
3                           1.77             7478.0   
4                            NaN             7498.0   

   daily_vaccinations_per_million  
0                             NaN  
1                          1205.0  
2                          1445.0  
3                          1525.0  
4                          1529.0  

[5 rows x 26 columns]

In [80]:
mergeCovStatePovVacts.state.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [81]:
state_ts = mergeCovStatePovVacts[['state', 'date', 'cases', 'deaths', 'male', 'female', 'population',
       'female_proportion', 'FIPStxt', 'Stabr', 
       'POVALL_2018', 'PCTPOVALL_2018', 'POV017_2018', 'PCTPOV017_2018',
       'MEDHHINC_2018', 'people_vaccinated', 'people_fully_vaccinated_per_hundred', 'total_vaccinations_per_hundred',
       'people_fully_vaccinated', 'people_vaccinated_per_hundred',
       'daily_vaccinations', 'daily_vaccinations_per_million']]

In [85]:
state_ts.head()

state        date   cases  deaths     male   female  population  \
0  Alabama  2021-01-12  407848    5573  2355799  2508881     4864680   
1  Alabama  2021-01-13  410995    5760  2355799  2508881     4864680   
2  Alabama  2021-01-14  414583    5945  2355799  2508881     4864680   
3  Alabama  2021-01-15  417528    6030  2355799  2508881     4864680   
4  Alabama  2021-01-16  420681    6119  2355799  2508881     4864680   

   female_proportion  FIPStxt Stabr  ...  POV017_2018  PCTPOV017_2018  \
0           0.515734     1000    AL  ...       255613            23.9   
1           0.515734     1000    AL  ...       255613            23.9   
2           0.515734     1000    AL  ...       255613            23.9   
3           0.515734     1000    AL  ...       255613            23.9   
4           0.515734     1000    AL  ...       255613            23.9   

   MEDHHINC_2018  people_vaccinated  people_fully_vaccinated_per_hundred  \
0          49881            70861.0                                 0.15   
1          49881            74792.0                                 0.19   
2          49881            80480.0                                  NaN   
3          49881            86956.0                                 0.28   
4          49881                NaN                                  NaN   

   total_vaccinations_per_hundred  people_fully_vaccinated  \
0                            1.59                   7270.0   
1                            1.71                   9245.0   
2                            1.88                      NaN   
3                            2.05                  13488.0   
4                             NaN                      NaN   

   people_vaccinated_per_hundred  daily_vaccinations  \
0                           1.45                 NaN   
1                           1.53              5906.0   
2                           1.64              7083.0   
3                           1.77              7478.0   
4                            NaN              7498.0   

   daily_vaccinations_per_million  
0                             NaN  
1                          1205.0  
2                          1445.0  
3                          1525.0  
4                          1529.0  

[5 rows x 22 columns]

In [ ]:
statelonlat = pd.read_csv('data/StateLonLat.csv')
statelonlat.head()

In [ ]:
mergeCovState2 = pd.merge(mergeCovState, statelonlat, how='left', on = 'state')

In [ ]:
lis = []
for str in  mergeCovState2['date']:
    lis.append(int(str[-1]))
    
lis

In [ ]:
mergeCovState2["daylastdigit"] = lis
mergeCovState10days = mergeCovState2[mergeCovState2["daylastdigit"]== 0 ]
mergeCovState10days.head()

In [ ]:
fig = px.scatter_geo(mergeCovState10days, lon = "longitude", lat = "latitude", hover_name='state', size="cases",
               animation_frame="date", projection='albers usa')
fig.show()

In [ ]:
lis = []
for str in  county_ts['date']:
    lis.append(int(str[-1]))
lis

In [ ]:
county_ts["daylastdigit"] = lis
county_ts10days = county_ts[county_ts["daylastdigit"] == 0 ]
county_ts10days.head()

In [ ]:
fig = px.scatter_geo(county_ts, lon = "long_x", lat = "lat_x", hover_name='county_x', size="cases",
               animation_frame="date", projection='albers usa', color = "Rural-urban_Continuum_Code_2013")
fig.update_layout(
    legend=dict(
        traceorder="reversed",
        title_font_family="Times New Roman",
        font=dict(
            family="Courier",
            size=9,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    )
)
fig.show()